In [13]:
import numpy as np
import pandas as pd
import pickle

In [ ]:
df=pd.read_csv("sentiment_analysis.csv")

In [ ]:
df.isna().sum()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
import string
import nltk
import re

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
sw=nltk.corpus.stopwords.words("english")
sw[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [ ]:
ps=nltk.PorterStemmer()

In [ ]:
def cleantext(txt):
  lst1="".join([i for i in txt if i not in string.punctuation])
  lst2=re.split('\W+',txt)
  lst3=[i for i in lst2 if i not in sw]
  lst4=[ps.stem(i) for i in lst3]
  return lst4
df["clean"]=df["text"].apply(lambda x : cleantext(x))
df.head()


In [ ]:
x=pd.DataFrame(df["clean"])
x.head()

In [ ]:
y=pd.DataFrame(df["label"])
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [ ]:
import tensorflow
from tensorflow import keras

In [ ]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(x_train["clean"])
x_train_seq=tokenizer.texts_to_sequences(x_train["clean"])
x_test_seq=tokenizer.texts_to_sequences(x_test["clean"])

In [ ]:
x_train_seq_padded=pad_sequences(x_train_seq,50)
x_test_seq_padded=pad_sequences(x_test_seq,50)

In [ ]:
from keras.layers import Dense,Embedding,LSTM
from keras.models import Sequential


In [ ]:
model_sq=Sequential()
model_sq.add(Embedding(len(tokenizer.index_word)+1,32))
model_sq.add(LSTM(32,dropout=0,recurrent_dropout=0))
model_sq.add(Dense(32,activation='relu'))
model_sq.add(Dense(1,activation='sigmoid'))
model_sq.summary()

In [ ]:
#compile the model
model_sq.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
#fit the RNN model
model_sq.fit(x_train_seq_padded,y_train['label'],batch_size=32,epochs=5,validation_data=(x_test_seq_padded,y_test))

In [ ]:
data="Hi there, I hope you are doing good! Take care and love"

In [ ]:
newdata=pd.DataFrame({"text":[s]})
newdata

In [ ]:
newdata["clean"]=newdata["text"].apply(lambda x:cleantext(x))
newdata.head()

In [ ]:
tokenizer.fit_on_texts(newdata["clean"])
x=tokenizer.texts_to_sequences(newdata["clean"])

In [ ]:
x_padded=pad_sequences(x,50)

In [ ]:
model_sq.predict(x_padded)

In [ ]:
model_sq.save("sentiment.h5",save_format='h5')

In [ ]:
model_load=tensorflow.keras.models.load_model("sentiment.h5")

In [ ]:
model_load.predict(x_padded)